In [ ]:
import os 
import numpy as np
import pandas as pd

import openpyxl

import matplotlib.pyplot as plt


import jou_gen as jg

from io import StringIO

import plotly.graph_objects as go

from numpy.polynomial.polynomial import Polynomial

import ansys.fluent.core as pyfluent

In [ ]:
folder_path = r'D:\ANSYS Fluent Projects\Hydraulics\V4.5\19MPE_woch\fluent'

N_junction = 38
N_channels = 304
Area_junction = 44.0414E-6

N_big_it = 25
N_it = 40

In [ ]:
def process_df(df,mdot_distrib_inlet):

    positive_count = (df['mdot'] > 0).sum()
    null_count = (df['mdot'] == 0).sum()

    if positive_count > 0:

        perc_to_adjust = ( 1 - mdot_distrib_inlet/-df[df['mdot'] < 0]['mdot'].sum() )
        perc_to_distribute = 0.01
        mdot_to_distribute = perc_to_distribute * -df[df['mdot'] < 0]['mdot'].sum()
        perc_to_remove = perc_to_adjust + perc_to_distribute

        df['mdot_updated'] = np.where(df['mdot'] < 0, (1-perc_to_remove)*df['mdot'], mdot_to_distribute / positive_count)
        df['mdot'] = df['mdot_updated'].abs()

    elif null_count > 0:

        perc_to_distribute = 0.05
        mdot_to_distribute = perc_to_distribute * -df[df['mdot'] < 0]['mdot'].sum()

        df['mdot_updated'] = np.where(df['mdot'] < 0, (1-perc_to_distribute)*df['mdot'], mdot_to_distribute / null_count)
        df['mdot'] = df['mdot_updated'].abs()

    else:
        df['mdot'] = df['mdot'].abs()

    df['mdot_ch_norm'] = df['mdot']/(df['mdot'].sum()/N_junction)
    df['new_input'] = df['mdot_ch_norm'] * (mdot_distrib_inlet/N_junction)

    return df

In [ ]:
solver_path = folder_path + '\\' + 'server_info-23956.txt'
solver = pyfluent.connect_to_fluent(server_info_filepath=solver_path)
tui = solver.tui

In [ ]:
with open(folder_path + '\\' + 'create_mdot_report_CPH.txt', "w") as file:
    file.write(jg.write_report_massflow(folder_path+'\\CPH\\'+'mdot_report.txt'))

In [ ]:
with open(folder_path + '\\' + 'create_sp_report_CPH.txt', "w") as file:
    file.write(jg.write_report_sp_prepared(folder_path+'\\CPH\\'+'sp_report.txt'))

In [ ]:
tui.file.read_journal('CPH\\create_mdot_report_CPH.txt')
tui.file.read_journal('CPH\\create_sp_report_CPH.txt')

In [ ]:
jg.convert_report(folder_path+'\\CPH','mdot_report.txt','mdot',output_file_name_wo_extension='mdot_report_IT1')
jg.convert_report(folder_path+'\\CPH','sp_report.txt','sp',output_file_name_wo_extension='sp_report_IT1')

In [ ]:
df = pd.read_csv(folder_path+'\\CPH\\mdot_report_IT1.csv')
df2 = pd.read_csv(folder_path+'\\CPH\\sp_report_IT1.csv')

merged_df = pd.merge(df, df2, on='Component')
dff = merged_df[merged_df['Component'].str.contains('coll_ch|distrib_ch')]
dff['Component'].astype(str)

coll_df = dff[dff['Component'].str.contains('coll_ch')].copy()
coll_df['index'] = coll_df['Component'].str.extract(r'coll_ch_(\d+)')
coll_df.dropna(subset='index', inplace=True)
coll_df['index'] = coll_df['index'].astype(int)
coll_df.sort_values(by='index', inplace=True)
coll_df.set_index('index', inplace=True, drop=True)

distrib_df = dff[dff['Component'].str.contains('distrib_ch')].copy()
distrib_df['index'] = distrib_df['Component'].str.extract(r'distrib_ch_(\d+)')
distrib_df.dropna(subset='index', inplace=True)
distrib_df['index'] = distrib_df['index'].astype(int)
distrib_df.sort_values(by='index', inplace=True)
distrib_df.set_index('index', inplace=True, drop=True)

In [ ]:
plt.plot(distrib_df.index,distrib_df['mdot'])
plt.plot(coll_df.index,coll_df['mdot'])

In [ ]:
coll_df['sp']

In [ ]:
plt.plot(distrib_df.index,distrib_df['sp'],label='distrib')
plt.plot(coll_df.index,coll_df['sp'],label='coll')
plt.plot(coll_df.index,coll_df['sp'] + 1411785 * (coll_df['mdot'] / 8),label='coll + 1411785 * (mdot / 8)')
plt.legend()

In [ ]:
distrib_df_P = process_df(distrib_df,merged_df[merged_df['Component'] == 'distrib_inlet']['mdot'].values[0])

A l'issue de l'itération N

Etape 1

On met le profil de débits obtenus en sortie de distributeur -> en entrée du collecteur
On met en condition à la limite en sortie du distributeur la valeur de pression mesurée en entrée du collecteur + coeff * débit mis en entrée collecteur, et un target mass flow rate = à 
l'opposé du débit mesuré en entrée collecteur

On fait tourner

Etape 2

On actualise les valeurs de pression en sortie du distributeur avec ce qu'on a obtenu en entrée collecteur + coeff * débit entrée collecteur

In [ ]:
string_list = []

for i in range(N_junction):
    string_list.append(jg.change_bc_velocity_inlet(f"coll_ch_{i+1}",f"( {{mdot_distrib_ch_{i+1}}} / rho_water ) / (44.0414E-6 [m^2])"))

jg.concatenate_and_write_to_file(string_list,folder_path+'\\CPH\\'+'change_bc_v_CPH_last_test.txt')

In [ ]:
### with target mass flow

string_list = []
for i in range(N_junction):
    sp_value = coll_df.loc[i+1,'sp'] + 1411785 * (distrib_df.loc[i+1,'new_input'] / 8)
    string_list.append(f"""
define/boundary-conditions/pressure-outlet distrib_ch_{i+1} yes no {sp_value} no no yes yes no no yes no {distrib_df.loc[i+1,'new_input']} no 5000000 no 1""")
jg.concatenate_and_write_to_file(string_list,folder_path + '\\CPH\\' + 'change_bc_po_non_uniform_target_CPH.txt')

In [ ]:
### without target mass flow

string_list = []
for i in range(N_junction):
    sp_value = coll_df.loc[i+1,'sp'] + 1411785 * (distrib_df.loc[i+1,'new_input'] / 8)
    string_list.append(f"""
define/boundary-conditions/pressure-outlet distrib_ch_{i+1} yes no {sp_value} no no yes yes no no no""")
jg.concatenate_and_write_to_file(string_list,folder_path + '\\CPH\\' + 'change_bc_po_non_uniform_without_target_CPH.txt')

In [ ]:
name_BC yes no value no no yes yes no no yes no "{{mdot_coll_ch_{i}}}" no 5000000 no 1

In [ ]:
### LAST TEST

string_list = []
for i in range(N_junction):
    string_list.append(f"""
define/boundary-conditions/pressure-outlet distrib_ch_{i+1} yes no "{{sp_coll_ch_{i+1}}} + 1411785 [Pa s / kg] * ( {{mdot_coll_ch_{i+1}}} / 8)" no no yes yes no no yes no "{{mdot_coll_ch_{i+1}}}" no 5000000 no 1""")
jg.concatenate_and_write_to_file(string_list,folder_path + '\\CPH\\' + 'change_bc_po_non_uniform_CPH_LAST_TEST.txt')

In [ ]:
tui.file.read_journal('CPH\\change_bc_v_CPH_last_test.txt')

In [ ]:
tui.file.read_journal('CPH\\change_bc_po_non_uniform_CPH_LAST_TEST.txt')